In [ ]:
import mlflow
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# get mlflow runs
experiment_id = '779465877152455589' # TODO update mlflow experiment ID if it changes (check mlruns directory)

mlflow.set_tracking_uri("../../mlruns")

runs = mlflow.search_runs(experiment_ids=[experiment_id])

failed_runs = len(runs[runs['status']=='FAILED'][['params.model']])
print("{} experiment runs failed ({}% of total)".format(failed_runs, failed_runs/len(runs)*100))

In [ ]:
# timestamp for saving figures, tables and other outputs from this experiment run
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print(timestamp)

In [ ]:
runs.to_csv(f'results-{timestamp}.csv', index=None)

In [ ]:
# cleanup the data fields for analysis
runs['trial'] = runs['params.datafile'].str.split('_').str[-2]
runs['value'] = runs['params.datafile'].str.split('_').str[-4].astype(float)
runs['variable'] = runs['params.datafile'].str.split('_').str[-5]
runs['C'] = runs['params.datafile'].str.split('_').str[-6]
runs['params.causal_distance'] = runs['params.causal_distance'].fillna('')
runs['params.inference_type'] = runs['params.inference_type'].fillna('')
runs['method'] = runs['params.model'] + runs['params.causal_distance'] + runs['params.inference_type']

In [ ]:
results = runs[['trial','variable','value','C','method','metrics.runtime_initialisation','metrics.runtime_main_training']]
results = results.groupby(['variable','value','method'])['metrics.runtime_initialisation','metrics.runtime_main_training'].agg(['mean', 'std']).reset_index()
for metric in 'metrics.runtime_initialisation','metrics.runtime_main_training':
    results["{}_fmt".format(metric)] = results.apply(lambda x: f"{x[(metric,'mean')]:.3f} ({x[(metric,'std')]:.3f})", axis=1)
results['metric'] = results['metrics.runtime_initialisation_fmt'] + '\n' + results['metrics.runtime_main_training_fmt']
results = results[['variable','value','method','metric']]
results = results.pivot(index=['variable','value'],columns='method',values='metric').reset_index()
results.to_csv(f'results-timing-{timestamp}.csv', index=None)
results